# The May-Leonhard Model

In this example, we come to a different type of model system, the three-species May-Leonhard Model. It is also sometimes called the cyclic Lotka-Volterra model for the simple reason that its three species A, B and C are interacting in a cyclic manner - A consumes B, B consumes C and C consumes A. In contrast to the closely related Rock-Paper-Scissors model, the May-Leonhard model does not incorporate reproduction in its predation reactions, but has independent reproduction processes for each species. The most striking feature of this model when run on a two-dimensional lattice is the emerging spiral structures. We will strive to reproduce these spirals here.  

To start, we again need to load the `pypop` library. We only load what we need so not to clutter our environment.

In [ ]:
from pypop import World, Species, BirthReaction, PredationReaction, Hop

Next, we set up the world. This involves setting up the species (the predator A and the prey B) and the reactions (B can give birth, A can die and predate upon B). The following are the parameters of the simulation:
* `size`: This is the size of the 2D lattice and is specified as a tuple of two integers (the width and the height) like so `(256, 256)`. It is a good idea to start small since larger lattices take more memory and more computational time to simulate.
* `sigma`: This is the predation rate of all three species, i.e. the rate with which A consumes B, B consumes C and C consumes A. For simplicity, we only look at the symmetric version wherein the rates are the same for all three species.
* `mu`: This is the reproduction rate of all three species.
* `h`: This is the hop rate, which in typically is left at a value of `1` since this will only rescale time. It could be varied for each species separately, such that one species moves slower than the others. Again, for simplicity's sake, we only look at the symmetric case.
* `rho`: This is the initial density of each of the three species. It specifies how many occupants of each species will be present on each lattice site on average.
* `T`: Specifies the end time of the simulation, i.e. how many Monte Carlo steps will be performed.
* `K`: This is the carrying capacity of the lattice sites. It determines how many particles a site can hold. We have this set to 1 here.

One crucial difference to the previous examples is that we introduce a carrying capacity into the system, and that this carrying capacity is set to one. Internally this changes the microscopic rules, such that interaction occurs across neighboring lattice sites rather than on the same site, and particles swap positions instead of simply hopping from lattice site to lattice site. 

In [ ]:
# The simulation parameters
size = (250, 250)
sigma: float = 0.1
mu: float = 0.1
h: float = 1.0
rho: float = 0.01
T: int = 1000
K: int = 1

# Predator and prey species
A = Species("A")
B = Species("B")
C = Species("C")

# The reactions
reactions = {
    A: [
        PredationReaction(A, B, sigma),
        BirthReaction(A, mu),
    ],
    B: [
        PredationReaction(B, C, sigma),
        BirthReaction(B, mu),
    ],
    C: [
        PredationReaction(C, A, sigma),
        BirthReaction(C, mu),
    ],
}

# Initialization of the world
world = World(
    size=size,
    initial_densities={A: rho, B: rho, C: rho},
    hops={A: Hop(A, h), B: Hop(B, h), C: Hop(C, h)},
    reactions=reactions,
    carrying_capacity=K,
)

Now we're ready to tun the simulations. We run `T` Monte Carlo steps (default is 1000). How long this takes will depend on the parameter settings above and the hardware this is being run on. With the default parameters on a typical machine it takes about three hours). This might be significantly more or less depending on the specific hardware.

Please note that the goal of the `PyPop` module is not computational efficiency but rather code clarity as it is an educational project. This could easily be made to run 100x faster if proper optimizations are applied, however this would sacrifice the readability of the module's code.

In [ ]:
from tqdm.auto import tqdm

# Initialize list that will hold a view of the lattice state at each step.
# The first entry shows the initial distribution of species site occupants.
numbers = [world.asarrays()]

# Iterate over `T` time steps
# `tqdm` is a tool to display a progress bar.
for _ in tqdm(range(T), smoothing=0, desc="Simulation progress"):
    # Run a single time step
    world.step()
    # Save the state of the lattice (distribution of occupants)
    numbers.append(world.asarrays())

After this has successfully run, we'd like to visualize the results. The code in the cell below displays an interactive graph where you can see the state of the lattice at each step (left side), together with a plot of the species densities over time (right side). 

In [ ]:
# Display matplotlib figures inline (not the interactive notebook extension)
%matplotlib inline
# Load modules
from matplotlib.figure import Figure
from ipywidgets import interact
import numpy as np
from IPython.display import display

# Initialize figure and axes
fig = Figure(figsize=(8, 4), dpi=150)
ax = fig.add_axes([0, 0, 0.5, 1])
# We don't want any decorations on the axes.
ax.axis('off')
plot = fig.add_axes([0.6, 0.1, 0.4, 0.8])

# Extract densities from species distribution arrays
N = size[0]*size[1]
densities = {
    species.name: [timepoint[species.name].sum()/N for timepoint in numbers]
    for species in world.species
}

# Plot the densities
for species, density in sorted(densities.items()):
    plot.plot(density, label=species)

# Label the plot
plot.set_yscale("log", base=10)
plot.set_ylabel("Species density [occupants/site]")
plot.set_xlabel("Time [MC steps]")

# This code displays the correct view of the lattice given the MC step
mappable = None
vline = None
def show_image(MC_step=0):
    global mappable, vline  # Hackish, but easy, avoid global if possible.

    # Load array of lattice occupant numbers
    arrays = numbers[MC_step]
    # Create a red and blue image
    # `image` is a WxHx3 image where the three channels at the end correspond to red-green-blue
    image = np.zeros((size[0], size[1], 3), dtype=np.uint8)
    # Make a pixel that has at least one predator red, at least one prey blue and purple if both are present.
    # Leave the green channel alone.
    image[:, :, 0] = 255*(arrays["A"] > 0)
    image[:, :, 1] = 255*(arrays["C"] > 0)
    image[:, :, 2] = 255*(arrays["B"] > 0)
    if mappable is None:
        # We need to create the mappable first, afterwards we can just load data into it.
        mappable = ax.imshow(image)
        # Similar for the vline indicating the MC step we're at.
        vline = plot.axvline(0, color="black")
    else:
        # Load data into mappable and vline.
        mappable.set_data(image)
        vline.set_xdata([MC_step])

    # Finally, display the updated figure.
    display(fig)

# THis lets us have a slider to select the MC step to display.
interact(show_image, MC_step=(0, len(numbers)-1))
None

After the initial transient growth in population of all three species, what you should see are cyclic activity fronts, which form localised spiral structures. They always occur in pairs, with opposite direction of rotation (but this can be hard to see when multiple spirals are close to each other).